# using to generate _implicit.filter.clean.txt

In [1]:
import pandas as pd
import numpy as np
import csv
import datetime 
import time
%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns
from src import settings
from sklearn import metrics

# retail_rocket_root_path = settings.DATA_ROOT_PATH + 'site_data/retail_rocket/'
retail_rocket_root_path = settings.DATA_ROOT_PATH + 'site_data/ml-1m/'

In [19]:
# khao sat tu bo data ban dau.
implicit_retail_rocket_df = pd.read_csv(retail_rocket_root_path + '_implicit.clean.txt', header=None, names=['uid', 'itemid', 'time'])
print(implicit_retail_rocket_df.head())

    uid     itemid  time
1  1193  978300760     5
1   661  978302109     3
1   914  978301968     3
1  3408  978300275     4
1  2355  978824291     5


In [16]:
# load list key for i2index
with open(retail_rocket_root_path + 'i2index.txt') as f:
    active_item_dict = {int(line.split(',')[0]): int(line.split(',')[1]) for line in f}
active_item_list = active_item_dict.keys()
print "Number of items: ",len(active_item_list)

# list key index from u2index.
with open(retail_rocket_root_path + 'u2index.txt') as f:
    active_user_dict = {int(line.split(',')[0]): int(line.split(',')[1]) for line in f}
active_user_list = active_user_dict.keys()
print "Number of users: ",len(active_user_list)

Number of items:  3706
Number of users:  6040


In [20]:
print "length before filter: ", len(implicit_retail_rocket_df)
# loai bo tuong tac ma user, item khong co trong active_user_list va active_item_list.
implicit_retail_rocket_df = implicit_retail_rocket_df[implicit_retail_rocket_df.uid.isin(active_user_list)]
print "length after filter 1: ", len(implicit_retail_rocket_df)
implicit_retail_rocket_df = implicit_retail_rocket_df[implicit_retail_rocket_df.itemid.isin(active_item_list)]
print "length after filter: ", len(implicit_retail_rocket_df)

length before filter:  1000209
length after filter 1:  1000209
length after filter:  0


In [54]:
implicit_retail_rocket_df = implicit_retail_rocket_df.groupby(["uid"]).apply(lambda x: x.sort_values(["uid","time"], ascending = True)).reset_index(drop=True)

In [55]:
implicit_retail_rocket_df[:20]

,uid,itemid,time
0,51,358388,1441495257511
1,51,198762,1441495318252
2,51,429304,1441495416363
3,51,198762,1441495582729
4,51,49967,1441495634886
5,51,279059,1441495684069
6,54,388096,1442338531705
7,54,283115,1442338665550
8,54,38965,1442338748248
9,54,319680,1442338841333


In [ ]:
out_file = retail_rocket_root_path + "_implicit.filter.clean.txt"
implicit_retail_rocket_df.to_csv(out_file, header=None, index=None, sep=',')

# using to generate _explicit.filter.clean.txt

In [57]:
explicit_retail_rocket_df = pd.read_csv(retail_rocket_root_path + '_explicit.clean.txt', header=None, names=['uid', 'itemid', 'time'])
print(explicit_retail_rocket_df.head())

      uid  itemid           time
0  287857    5206  1433223236124
1  158090   10572  1433221078505
2  599528  356475  1433221941632
3  599528  356475  1433222276276
4  105775  312728  1433220880956


In [58]:
print "length before filter: ", len(explicit_retail_rocket_df)
# loai bo tuong tac ma user, item khong co trong active_user_list va active_item_list.
explicit_retail_rocket_df = explicit_retail_rocket_df[explicit_retail_rocket_df.uid.isin(active_user_list)]
explicit_retail_rocket_df = explicit_retail_rocket_df[explicit_retail_rocket_df.itemid.isin(active_item_list)]
print "length after filter: ", len(explicit_retail_rocket_df)

length before filter:  88886
length after filter:  39939


In [59]:
explicit_retail_rocket_df = explicit_retail_rocket_df.groupby(["uid"]).apply(lambda x: x.sort_values(["uid","time"], ascending = True)).reset_index(drop=True)

In [60]:
explicit_retail_rocket_df[:20]

,uid,itemid,time
0,155,452082,1441951937583
1,155,181405,1441951967871
2,155,41882,1441952264613
3,155,442601,1441952265950
4,155,389974,1442032680026
5,172,10034,1439599816912
6,172,465522,1439601219691
7,172,465522,1439601228324
8,172,465522,1439602141230
9,172,10034,1439602141686


In [ ]:
out_file = retail_rocket_root_path + "_explicit.filter.clean.txt"
explicit_retail_rocket_df.to_csv(out_file, header=None, index=None, sep=',')

# merge to type data

In [39]:
import pandas as pd
from src import settings
retail_rocket_root_path = settings.DATA_ROOT_PATH + 'site_data/retail_rocket/'
file1 = open(retail_rocket_root_path+"z_temp_ratings_tinytest.txt","r")

def Sort(sub_li):   
    # reverse = None (Sorts in Ascending order) 
    # key is set to sort using second element of  
    # sublist lambda has been used 
    sub_li.sort(key = lambda x: x[1]) 
    return sub_li

interact_item_sequences = {} # cac chuoi item sequences tuong ung cho moi
interact_sequences ={} 
# thong tin tuong ung voi cac sp: 
#0: chi co tuong tac implicit 
#1: chi co tuong tac explicit 
#2: co ca implicit & explicit

for line in file1.readlines():
    line = line.strip()
    elements = line.split("|")
    # user id
    uid = elements[0].strip() 
    # explicit interact
    explicit_interact = elements[2].strip()
    
    if explicit_interact == '-1': # khong co tuong tac explicit
        # xu ly implicit interact.
        implicit_interacts = elements[1].strip()
        implicit_interacts = implicit_interacts[2:-2] # loai bo 2 phan tu dau va cuoi

        list_item = []
        for implicit_pair in implicit_interacts.split("],"):
            implicit_pair = implicit_pair.strip()  
            if '[' in implicit_pair:
                implicit_pair = implicit_pair[1:]
            itemid = implicit_pair.split(",")[0].strip()[1:-1]
            time = implicit_pair.split(",")[1].strip()[1:-1]
            list_item.append(int(itemid))
            print itemid + "-" + time
        # tong hop lai vao dict[uid]
        interact_item_sequences[uid] = list_item
        interact_sequences[uid] = [0] * len(list_item) # mang gom list cac phan tu = 0
    
    else:  # neu co tuong tac explicit
        # xu ly implicit interact.
        implicit_interacts = elements[1].strip()
#         print "ban dau:"
#         print implicit_interacts
        implicit_interacts = implicit_interacts[2:-2] # loai bo 2 phan tu dau va cuoi

        list_implicit = []
        list_item_implicit =[]
        for implicit_pair in implicit_interacts.split("],"):
            implicit_pair = implicit_pair.strip()  
            if '[' in implicit_pair:
                implicit_pair = implicit_pair[1:]
            itemid = implicit_pair.split(",")[0].strip()[1:-1]
            time = implicit_pair.split(",")[1].strip()[1:-1]
            
            list_implicit.append([itemid+'i', int(time)])
#         print "Sau do:"
#         print Sort(list_implicit)
        
        # with explicit interact.
        explicit_interacts = elements[2].strip()
        explicit_interacts = explicit_interacts[2:-2] # loai bo 2 phan tu dau va cuoi        
        list_explicit = []
        list_item_explicit =[]
        for explicit_pair in explicit_interacts.split("],"):
            explicit_pair = explicit_pair.strip()  
            if '[' in explicit_pair:
                explicit_pair = explicit_pair[1:]
            itemid = explicit_pair.split(",")[0].strip()[1:-1]
            time = explicit_pair.split(",")[1].strip()[1:-1]
            
            list_explicit.append([itemid+'e', int(time)])
            
        total_list = list_implicit + list_explicit
        
        # list item
        total_item =[]
        for _ in total_list:
            if int(_[0][:-1]) not in total_item:
                total_item.append(int(_[0][:-1]))
    
        # sorting by time:
        sorted_list = Sort(total_list)
        
        # tong hop lai vao dict[uid]
        if len(total_item) == len(sorted_list):# neu so item xh == so ban ghi
            for element in sorted_list:
                interact_item_sequences[uid].append(int(element[0][:-1]))
                if element[0][-1] == 'i':
                    interact_sequences[uid].append(0)
                else:
                    interact_sequences[uid].append(1)
        
        # neu ngan hon, hay tuc la co 1 item duoc tuong tac tai cac thoi diem khac nhau.
        # list_item of sorted list (include i and e)
        print "Sorted list:"
        for _ in sorted_list:
            print _
            
        list_item_with_status =[]
        list_item_without_status =[]
        for _ in sorted_list:
            list_item_with_status.append(_[0])
            list_item_without_status.append(int(_[0][:-1]))
            
        if len(total_item) < len(sorted_list):
            print "VAO"
            temp_add_list_with_status =[]
            new_sort =[]
            for element in sorted_list:
                if element[0][-1] == 'i': # neu la implicit
                    new_sort.append([element[0], 0])
                    
                elif element[0][-1] == 'e': # neu la explicit
                    if temp_add_list_with_status.count(element[0].replace(element[0][-1],'i')) > 0:
#                         print "TEMP STT:",temp_add_list_with_status
                        # ma da co tuong tac implicit truoc do
                        # cap nhat thang implicit gan nhat tu 0 thanh 2.
                        for _ in reversed(new_sort):
                            if _[0] == element[0].replace(element[0][-1],'i'):
                                _[1] = 2
                                break
                    else:
                        new_sort.append([element[0], 1])        
                temp_add_list_with_status.append(element[0])
            print "New sorted:"
            for _ in new_sort:
                print _
#             print interact_item_sequences[uid]
#             for _ in new_sort:# ghi vao list interact
#                 interact_item_sequences[uid].append(int(_[0][:-1]))
#                 interact_sequences[uid].append(_[1])
    
    print interact_item_sequences
    print interact_sequences
            
            
            

358388-1441495257511
198762-1441495318252
429304-1441495416363
198762-1441495582729
49967-1441495634886
279059-1441495684069
{'51': [358388, 198762, 429304, 198762, 49967, 279059]}
{'51': [0, 0, 0, 0, 0, 0]}
Sorted list:
['212378i', 1437417563381]
['134852e', 1437420807417]
['134852e', 1437420815558]
['254290e', 1437420959995]
['98840e', 1437421273477]
['98840i', 1437421422889]
['270332i', 1437421551203]
['115326i', 1437421658021]
['115326e', 1437421684384]
['115326i', 1437421732759]
['270332i', 1437422087771]
['270332e', 1437422123209]
['90420i', 1437503804949]
['129898i', 1437512337045]
VAO
New sorted:
['212378i', 0]
['134852e', 1]
['134852e', 1]
['254290e', 1]
['98840e', 1]
['98840i', 0]
['270332i', 0]
['115326i', 2]
['115326i', 0]
['270332i', 2]
['90420i', 0]
['129898i', 0]
{'51': [358388, 198762, 429304, 198762, 49967, 279059]}
{'51': [0, 0, 0, 0, 0, 0]}


In [2]:
from src import settings
root_path = settings.DATA_ROOT_PATH + 'site_data/retail_rocket/ratings_train.txt'
file_data = open(root_path,"r")
for line in file_data.read_lines():
    print line
    break

AttributeError: 'file' object has no attribute 'read_lines'